# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san pedro pochutla,15.7333,-96.4667,87.89,43,0,11.30,MX,1707677871
1,1,uliastay,47.7417,96.8444,13.41,75,100,6.02,MN,1707677872
2,2,port-aux-francais,-49.3500,70.2167,47.53,99,100,36.06,TF,1707677873
3,3,waitangi,-43.9535,-176.5597,53.82,76,100,3.00,NZ,1707677874
4,4,haiku-pauwela,20.9219,-156.3051,75.04,82,0,4.61,US,1707677875


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

base_url = "https://api.geoapify.com/v1/geocode/search"

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600    
)



# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrow1_df = city_data_df[(city_data_df['Humidity'] >=50) & (city_data_df['Humidity']<=75)]
narrow2_df = narrow1_df[(narrow1_df['Cloudiness'] <=50)]
narrow3_df = narrow2_df[(narrow2_df['Max Temp']<= 90) & (narrow2_df['Max Temp'] >= 50)]


# Drop any rows with null values
narrow3_df = narrow3_df.dropna()

# Display sample data
narrow3_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,santiago de cao,-7.9589,-79.2392,78.93,73,13,12.66,PE,1707677880
13,13,brisas de zicatela,15.8369,-97.0419,83.75,60,0,9.51,MX,1707677886
21,21,hermanus,-34.4187,19.2345,70.81,53,11,8.25,ZA,1707677894
29,29,edinburgh of the seven seas,-37.0676,-12.3116,64.20,70,2,15.66,SH,1707677902
32,32,olinda,-8.0089,-34.8553,87.53,66,40,18.41,BR,1707677906


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow3_df[["City", "Country", "Lat", 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,santiago de cao,PE,-7.9589,-79.2392,73,
13,brisas de zicatela,MX,15.8369,-97.0419,60,
21,hermanus,ZA,-34.4187,19.2345,53,
29,edinburgh of the seven seas,SH,-37.0676,-12.3116,70,
32,olinda,BR,-8.0089,-34.8553,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
 # Set base URL
base_url = "https://api.geoapify.com/v2/places?categories=accommodation.hotel&"


params = {
    "limit":limit,
    "apiKey": geoapify_key
    }


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filters"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
        
   
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
santiago de cao - nearest hotel: Hostal El Encanto
brisas de zicatela - nearest hotel: Casa de Olas
hermanus - nearest hotel: Aloe guest house
edinburgh of the seven seas - nearest hotel: No hotel found
olinda - nearest hotel: Fidji Motel
east london - nearest hotel: No hotel found
weno - nearest hotel: High Tide Hotel
punta arenas - nearest hotel: Hotel Lacolet
kingston - nearest hotel: Aphrodite Hotel
san antonio - nearest hotel: Riverwalk Plaza Hotel and Suites
al jawf - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
acapulco de juarez - nearest hotel: Hotel del Valle
celestun - nearest hotel: Hotel Flamingo Playa
nueve de julio - nearest hotel: Hotel Madrid
georgetown - nearest hotel: Page 63 hostel
hadibu - nearest hotel: Makka Hotel Bargaal
nouadhibou - nearest hotel: Hotel valencia
shanwei - nearest hotel: 汕尾保利希尔顿逸林酒店
touros - nearest hotel: Pousada Atlântico
arraial do cabo - nearest hotel: No hotel found
serra - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
8,santiago de cao,PE,-7.9589,-79.2392,73,Hostal El Encanto
13,brisas de zicatela,MX,15.8369,-97.0419,60,Casa de Olas
21,hermanus,ZA,-34.4187,19.2345,53,Aloe guest house
29,edinburgh of the seven seas,SH,-37.0676,-12.3116,70,No hotel found
32,olinda,BR,-8.0089,-34.8553,66,Fidji Motel
...,...,...,...,...,...,...
537,es castell,ES,39.8795,4.2854,62,Hotel Agamenon
550,mormugao,IN,15.4000,73.8000,69,Hotel Rukhmini
557,gebeit,SD,21.0667,36.3167,56,Sudan Red Sea Resort
558,ponnani,IN,10.7670,75.9252,73,Hotel Salkara


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
# we're looking for hotels, so get rid of all entries where we couldn't find one within 10 klicks

hotel_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == 'No hotel found'].index)

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,santiago de cao,PE,-7.9589,-79.2392,73,Hostal El Encanto
13,brisas de zicatela,MX,15.8369,-97.0419,60,Casa de Olas
21,hermanus,ZA,-34.4187,19.2345,53,Aloe guest house
32,olinda,BR,-8.0089,-34.8553,66,Fidji Motel
56,weno,FM,7.4515,151.8468,75,High Tide Hotel
...,...,...,...,...,...,...
537,es castell,ES,39.8795,4.2854,62,Hotel Agamenon
550,mormugao,IN,15.4000,73.8000,69,Hotel Rukhmini
557,gebeit,SD,21.0667,36.3167,56,Sudan Red Sea Resort
558,ponnani,IN,10.7670,75.9252,73,Hotel Salkara


In [22]:
%%capture --no-display

# Configure the map plot
base_url = "https://api.geoapify.com/v1/geocode/search"

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols=['Hotel Name','Country'],
    frame_width = 800,
    frame_height = 600    
)



# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)